In [1]:
# Import required libraries
import pandas as pd

# Load the dataset
df_operational = pd.read_csv("Merged_Superstore.csv")

# Remove any duplicate rows (based on Order ID and Product ID)
df_operational.drop_duplicates(subset=['Order ID', 'Product ID'], inplace=True)

# Function to fill missing postal codes based on city reference
def fill_missing_postal(row):
    if pd.isnull(row['Postal Code']) and row['City'] in postal_ref:
        return postal_ref[row['City']]
    return row['Postal Code']

# Create a reference dictionary of cities and their postal codes
postal_ref = df_operational.dropna(subset=['Postal Code']).groupby('City')['Postal Code'].first().to_dict()

# Fill in missing Postal Code values using the function
df_operational['Postal Code'] = df_operational.apply(fill_missing_postal, axis=1)

# Save cleaned data to CSV
df_operational.to_csv("Cleaned_Operational_Report.csv", index=False)

print("Data has been cleaned and saved to 'Cleaned_Operational_Report.csv'")


Data has been cleaned and saved to 'Cleaned_Operational_Report.csv'


In [2]:
# Export each year's data to a separate CSV file
for year, df in dataframes_by_year.items():
    csv_filename = f"Superstore_Data_{year}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Data for {year} saved to {csv_filename}")



NameError: name 'dataframes_by_year' is not defined

In [4]:
import pandas as pd
import sqlite3

# Load the "Cleaned_Operational_Report.csv" file into a DataFrame
df = pd.read_csv("Cleaned_Operational_Report.csv")

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Write the data to an SQLite table named "cleaned_data"
df.to_sql('cleaned_data', conn, index=False, if_exists='replace')

# Adjusted query to extract data sorted by "Order Date", excluding the specified columns
query_final_adjusted = """
SELECT 
    `Order ID`, `Order Date`, `Ship Date`, `Ship Mode`, `Customer Name`, 
    Segment, City, State, `Postal Code`, Region, Category, 
    Sales, Quantity, Discount, Profit, Returned
FROM cleaned_data
ORDER BY `Order Date`
"""

# Execute the adjusted query and store the result in a DataFrame
df_final_adjusted_extracted = pd.read_sql(query_final_adjusted, conn)


In [5]:
# Display the first few rows of the final extracted data
print(df_final_adjusted_extracted.head())


         Order ID  Order Date   Ship Date       Ship Mode  Customer Name  \
0  CA-2018-103800  2018-01-03  2018-01-07  Standard Class  Darren Powers   
1  CA-2018-112326  2018-01-04  2018-01-08  Standard Class  Phillina Ober   
2  CA-2018-112326  2018-01-04  2018-01-08  Standard Class  Phillina Ober   
3  CA-2018-112326  2018-01-04  2018-01-08  Standard Class  Phillina Ober   
4  CA-2018-141817  2018-01-05  2018-01-12  Standard Class     Mick Brown   

       Segment          City         State  Postal Code   Region  \
0     Consumer       Houston         Texas      77095.0  Central   
1  Home Office    Naperville      Illinois      60540.0  Central   
2  Home Office    Naperville      Illinois      60540.0  Central   
3  Home Office    Naperville      Illinois      60540.0  Central   
4     Consumer  Philadelphia  Pennsylvania      19143.0     East   

          Category    Sales  Quantity  Discount   Profit  Returned  
0  Office Supplies   16.448         2       0.2   5.5512         

In [6]:
# Calculate the delivery days
df_final_adjusted_extracted['Delivery Days'] = (pd.to_datetime(df_final_adjusted_extracted['Ship Date']) 
                                               - pd.to_datetime(df_final_adjusted_extracted['Order Date'])).dt.days

# Display the first few rows to check the new column
print(df_final_adjusted_extracted.head())


         Order ID  Order Date   Ship Date       Ship Mode  Customer Name  \
0  CA-2018-103800  2018-01-03  2018-01-07  Standard Class  Darren Powers   
1  CA-2018-112326  2018-01-04  2018-01-08  Standard Class  Phillina Ober   
2  CA-2018-112326  2018-01-04  2018-01-08  Standard Class  Phillina Ober   
3  CA-2018-112326  2018-01-04  2018-01-08  Standard Class  Phillina Ober   
4  CA-2018-141817  2018-01-05  2018-01-12  Standard Class     Mick Brown   

       Segment          City         State  Postal Code   Region  \
0     Consumer       Houston         Texas      77095.0  Central   
1  Home Office    Naperville      Illinois      60540.0  Central   
2  Home Office    Naperville      Illinois      60540.0  Central   
3  Home Office    Naperville      Illinois      60540.0  Central   
4     Consumer  Philadelphia  Pennsylvania      19143.0     East   

          Category    Sales  Quantity  Discount   Profit  Returned  \
0  Office Supplies   16.448         2       0.2   5.5512        

In [9]:
# Rearrange the columns
column_order = ['Order Date', 'Region', 'State', 'City', 'Category', 'Segment', 'Sales', 'Quantity', 'Discount', 'Profit', 'Returned', 'Delivery Days']
df_final_adjusted_extracted = df_final_adjusted_extracted[column_order]

# Add COGS and Margin columns
df_final_adjusted_extracted['COGS'] = df_final_adjusted_extracted['Sales'] - df_final_adjusted_extracted['Profit']
df_final_adjusted_extracted['Margin'] = (df_final_adjusted_extracted['Profit'] / df_final_adjusted_extracted['Sales']) * 100

# Export the DataFrame to a CSV file
df_final_adjusted_extracted.to_csv("Final_Superstore_Data.csv", index=False)
print("Data exported to 'Final_Superstore_Data.csv'")


Data exported to 'Final_Superstore_Data.csv'


<ipython-input-9-9e2f9677eb32>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_adjusted_extracted['COGS'] = df_final_adjusted_extracted['Sales'] - df_final_adjusted_extracted['Profit']


In [23]:
import pandas as pd
import sqlite3

# Load the "Final_Superstore_Data.csv" file into a DataFrame
df = pd.read_csv("Final_Superstore_Data.csv")

# Connect to an SQLite database
conn = sqlite3.connect(':memory:')

# Write the DataFrame to an SQLite table
df.to_sql('superstore_data', conn, index=False, if_exists='replace')

# Add 'Year' and 'Quarter' columns to the DataFrame based on 'Order Date'
df['Year'] = pd.to_datetime(df['Order Date']).dt.year
df['Quarter'] = pd.to_datetime(df['Order Date']).dt.quarter

# Update the SQLite table with the modified DataFrame
df.to_sql('superstore_data', conn, index=False, if_exists='replace')

# Calculate the KPIs year by year
yearly_kpis_query = """
SELECT 
    Year,
    SUM(Sales) as Total_Sales,
    COUNT(*) as Total_Orders,
    SUM(CASE WHEN Returned = 'Yes' THEN 1 ELSE 0 END) as Total_Returns,
    (SUM(Sales) / COUNT(*)) as Sales_Performance,
    (SUM(Sales) / ?) * 100 as Sales_Target_Attainment,
    (SUM(CASE WHEN Returned = 'Yes' THEN 1 ELSE 0 END) * 1.0 / COUNT(*)) * 100 as Return_Rate
FROM superstore_data
GROUP BY Year
ORDER BY Year
"""
yearly_kpis = pd.read_sql(yearly_kpis_query, conn, params=(avg_sales,))

# Calculate the KPIs quarter by quarter
quarterly_kpis_query = """
SELECT 
    Year,
    Quarter,
    SUM(Sales) as Total_Sales,
    COUNT(*) as Total_Orders,
    SUM(CASE WHEN Returned = 'Yes' THEN 1 ELSE 0 END) as Total_Returns,
    (SUM(Sales) / COUNT(*)) as Sales_Performance,
    (SUM(Sales) / ?) * 100 as Sales_Target_Attainment,
    (SUM(CASE WHEN Returned = 'Yes' THEN 1 ELSE 0 END) * 1.0 / COUNT(*)) * 100 as Return_Rate
FROM superstore_data
GROUP BY Year, Quarter
ORDER BY Year, Quarter
"""
quarterly_kpis = pd.read_sql(quarterly_kpis_query, conn, params=(avg_sales,))

# Format the KPIs as percentages with two decimal places
yearly_kpis['Sales_Performance'] = yearly_kpis['Sales_Performance'].map("{:,.2f}%".format)
yearly_kpis['Sales_Target_Attainment'] = yearly_kpis['Sales_Target_Attainment'].map("{:,.2f}%".format)
yearly_kpis['Return_Rate'] = yearly_kpis['Return_Rate'].map("{:,.2f}%".format)

quarterly_kpis['Sales_Performance'] = quarterly_kpis['Sales_Performance'].map("{:,.2f}%".format)
quarterly_kpis['Sales_Target_Attainment'] = quarterly_kpis['Sales_Target_Attainment'].map("{:,.2f}%".format)
quarterly_kpis['Return_Rate'] = quarterly_kpis['Return_Rate'].map("{:,.2f}%".format)

# Export the KPIs to CSV files
yearly_kpis.to_csv("Adjusted_Yearly_KPIs.csv", index=False)
quarterly_kpis.to_csv("Adjusted_Quarterly_KPIs.csv", index=False)

print("KPIs exported to 'Adjusted_Yearly_KPIs.csv' and 'Adjusted_Quarterly_KPIs.csv'")


KPIs exported to 'Adjusted_Yearly_KPIs.csv' and 'Adjusted_Quarterly_KPIs.csv'


/Users/kei/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [29]:
# Calculate Sales Targets for Yearly KPIs
yearly_kpis['Sales_Target'] = 0
for i in range(1, len(yearly_kpis)):
    yearly_kpis.loc[i, 'Sales_Target'] = yearly_kpis.loc[i-1, 'Total_Sales'] * 1.1

# Calculate Sales Targets for Quarterly KPIs
quarterly_kpis['Sales_Target'] = 0
for i in range(1, 4):
    quarterly_kpis.loc[i, 'Sales_Target'] = quarterly_kpis.loc[i-1, 'Total_Sales'] * 1.1
for i in range(4, len(quarterly_kpis)):
    quarterly_kpis.loc[i, 'Sales_Target'] = (quarterly_kpis.loc[i-4:i-1, 'Total_Sales'].sum()) / 4

# Recalculate Sales Target Attainment using the new Sales Targets
yearly_kpis['Sales_Target_Attainment'] = (yearly_kpis['Total_Sales'] / yearly_kpis['Sales_Target']) * 100
quarterly_kpis['Sales_Target_Attainment'] = (quarterly_kpis['Total_Sales'] / quarterly_kpis['Sales_Target']) * 100

# Format the newly calculated 'Sales_Target_Attainment' as percentages with two decimal places
yearly_kpis['Sales_Target_Attainment'] = yearly_kpis['Sales_Target_Attainment'].apply(lambda x: f"{x:.2f}%")
quarterly_kpis['Sales_Target_Attainment'] = quarterly_kpis['Sales_Target_Attainment'].apply(lambda x: f"{x:.2f}%")

# Export the KPIs to CSV files
yearly_kpis.to_csv("Final_Adjusted_Yearly_KPIs.csv", index=False)
quarterly_kpis.to_csv("Final_Adjusted_Quarterly_KPIs.csv", index=False)

yearly_kpis, quarterly_kpis



(   Year   Total_Sales  Total_Orders  Total_Returns Sales_Performance  \
 0  2018  5.754557e+05          2441            599           235.75%   
 1  2019  5.798728e+05          2522            583           229.93%   
 2  2020  7.152204e+05          3179            789           224.98%   
 3  2021  1.031128e+06          4278           1255           241.03%   
 
   Sales_Target_Attainment Return_Rate  Sales_Target  
 0                    inf%      24.54%       0.00000  
 1                  91.61%      23.12%  633001.26131  
 2                 112.13%      24.82%  637860.08770  
 3                 131.06%      29.34%  786742.41030  ,
     Year  Quarter  Total_Sales  Total_Orders  Total_Returns Sales_Performance  \
 0   2018        1   74911.8200           306             34           244.81%   
 1   2018        2   88506.1276           414             37           213.78%   
 2   2018        3  214885.2383           818            326           262.70%   
 3   2018        4  197152.50